# Notebook to analyze and display scRNAseq data


### Load important modules

In [ ]:
# Standard modules
import numpy as np
import os
import pandas as pd 
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy import linalg
from plotly.subplots import make_subplots
import ipywidgets as widgets
import logging
from sklearn import linear_model
from multiprocessing import Pool
import plotly.express as px
import scipy.cluster.hierarchy as sch
from numba import njit
from umap import UMAP
from sklearn.cluster import KMeans
import copy
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import ward, fcluster, leaves_list, optimal_leaf_ordering
import subprocess

# Move to root directory for easier module handling
os.chdir("../..")
print(os.listdir("."))

#LBAE imports
from modules.maldi_data import MaldiData
from modules.figures import Figures
from modules.atlas import Atlas
from modules.launch import Launch
from modules.storage import Storage


# multithreading/multiprocessing
from multiprocessing import Pool
import multiprocessing
from threadpoolctl import threadpool_limits

# set thread limit
threadpool_limits(16)


### Load LBAE objects

In [ ]:
path_data = "data/whole_dataset/"
path_annotations = "data/annotations/"
path_db = "data/app_data/data.db"

# Load shelve database
storage = Storage(path_db)

# Load data
data = MaldiData(path_data, path_annotations)

# Load Atlas and Figures objects. At first launch, many objects will be precomputed and shelved in
# the classes Atlas and Figures.
atlas = Atlas(data, storage, resolution=25, sample = True)
figures = Figures(data, storage, atlas, sample = True)

#### Load genes and lipids arrays 

In [ ]:
array_exp_lipids = np.load('data/scRNAseq/array_exp_lipids.npy')
array_exp_genes = np.load('data/scRNAseq/array_exp_genes.npy')
l_name_lipids = np.load('data/scRNAseq/array_name_lipids.npy').tolist()
l_genes = np.load('data/scRNAseq/array_name_genes.npy').tolist()
xmol, ymol, zmol = np.load('data/scRNAseq/array_coordinates.npy')


In [ ]:
print(array_exp_genes.shape, array_exp_lipids.shape)

#### Make an elatic net regression to explain lipid expression in terms of combination gene expression

In [ ]:
# Define regression as a function for parallelization
def compute_regression(index_lipid):
    print(index_lipid)
    clf = linear_model.ElasticNet(fit_intercept = True, alpha=0.2, positive = False)#linear_model.Lasso(alpha=0.001)##
    clf.fit(array_exp_genes, array_exp_lipids[:,index_lipid])
    return [clf.coef_, clf.score(array_exp_genes, array_exp_lipids[:,index_lipid])]

# Parallelize
#with Pool(processes=16) as pool:
l_lipid_indices = list(range(array_exp_lipids.shape[1]))
l_res =  [x for x in map(compute_regression, l_lipid_indices[:])]

ll_coef = []
l_score = []
for res in l_res:
    ll_coef.append(res[0])
    l_score.append(res[1])

print(l_score)

In [ ]:
print(l_score)

In [ ]:
array_coef = np.array(ll_coef)

In [ ]:
print(np.sum([1 for x in l_res[0][0] if abs(x)>0]))

In [ ]:
# Remove genes that do not explain the expression of any lipids
l_to_keep = []
for idx, col in enumerate(array_coef.T):
    if np.sum([1 for x in col if abs(x)>0])>15:
        l_to_keep.append(idx)
        
array_coef = array_coef[:, l_to_keep]
array_genes = np.array(l_genes)[l_to_keep]

In [ ]:
print(array_coef.shape)

In [ ]:
# Compute correlation on rows
pairwise_distances = sch.distance.pdist(array_coef)
linkage = sch.linkage(pairwise_distances, method='ward')
cluster_distance_threshold = pairwise_distances.max()/2
idx_to_cluster_array = sch.fcluster(linkage, cluster_distance_threshold, 
                                    criterion='distance')
idx = np.argsort(idx_to_cluster_array)
    
corr_array_clustered = array_coef[idx, :]

# Compute correlation on columns
pairwise_distances = sch.distance.pdist(corr_array_clustered.T)
linkage = sch.linkage(pairwise_distances, method='ward')
cluster_distance_threshold = pairwise_distances.max()/2
idx_to_cluster_array = sch.fcluster(linkage, cluster_distance_threshold, 
                                    criterion='distance')
idx = np.argsort(idx_to_cluster_array)
    
corr_array_clustered = corr_array_clustered[:, idx]



In [ ]:
print(corr_array_clustered.shape)

In [ ]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=True,horizontal_spacing=0.)


g1 = go.Heatmap(z = corr_array_clustered, colorscale='RdBu', x=array_genes, y = l_name_lipids, zmin = -0.002, zmax = 0.002, colorbar=dict(title="Coef value", x=1.1 ))
g2 = go.Heatmap(z = np.array([l_score]).T, colorscale='mint', y = l_name_lipids, zmin = 0., zmax = 0.6, colorbar=dict(title="R2 score", x=1.05 ))


fig.append_trace(g1, row=1, col=1)
fig.append_trace(g2, row=1, col=2)



# edit axis labels
fig['layout']['xaxis']['title']="Genes"
fig['layout']['yaxis']['title']='Lipids'
#fig['layout']['xaxis2']['title']='R2 score'
fig['layout']['xaxis2']['side']='top'

fig.update_xaxes(showticklabels=False, row=1, col=2)
fig.update_yaxes(showticklabels=False, row=1, col=2)

fig['layout']['yaxis2']['scaleanchor']='x2'
fig['layout']['yaxis']['scaleanchor']='x'

fig['layout']['xaxis']['domain'] = [0, 0.98]
fig['layout']['xaxis2']['domain'] = [0.99, 1]
#fig.update_traces(showscale=False)
fig.update_xaxes(tickangle=45)
#fig.update_layout(coloraxis_colorbar_x=-0.15)

    


fig.update_layout(
    width=int(1520),
    height=int(700),
    font_size=7,
    title_font_size=12,
    title={
        'text' : 'Elastic Net regression factors (MALDI lipids data with gene expression as predictors)',
        'y':0.92,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')


fig.show()
fig.write_image("notebooks/scRNAseq/heatmap.pdf")


#### Make an interactive widget displaying a clustergram of lipid and gene expression when one clicks on the corresponding dot

In [ ]:
# Get root figure
root_data = figures._storage.return_shelved_object(
    "figures/3D_page",
    "volume_root",
    force_update=False,
    compute_function=figures.compute_3D_root_volume,
)

# Get scatter figure for the scRNAseq spots
scatter = go.Scatter3d(
    x=xmol,
    y=ymol,
    z=zmol,
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8
    ),
)


root_data['hoverinfo']= 'skip'
scatter['hoverinfo']= "all"

In [ ]:
plot_widget = True
f = None
if plot_widget:
    fig = make_subplots(rows=1, cols=6, specs=[[{'type': 'surface', "colspan": 4},  {'type': 'xy'}, {'type': 'xy'}, {'type': 'xy'}, {'type': 'xy'}, {'type': 'xy'}]])
    fig.add_trace(root_data,row=1, col=1)
    fig.add_trace(scatter,row=1, col=1)

    n_to_keep = 53

    # Sort array according to most expressed genes and lipid for spot 0
    index_init = 0
    array_init_genes, l_genes_init = zip(*sorted(zip(array_exp_genes[index_init:index_init+1,:].T, l_genes), reverse=True))
    array_init_genes = np.array(array_init_genes)[:n_to_keep]
    l_genes_init = l_genes_init[:n_to_keep]
    array_init_lipids, l_lipids_init = zip(*sorted(zip(array_exp_lipids[index_init:index_init+1,:].T, l_name_lipids), reverse=True))
    array_init_lipids = np.array(array_init_lipids)[:n_to_keep]
    l_lipids_init = l_lipids_init[:n_to_keep]

    g1 = go.Heatmap(z = array_init_genes, colorscale='mint', y=l_genes_init)#, xaxis="x2", yaxis="y2", )
    g2 = go.Heatmap(z = array_init_lipids, colorscale='mint', y = l_lipids_init )#, xaxis="x3", yaxis="y3", )


    fig.add_trace(g1, row=1, col=5)
    fig.add_trace(g2, row=1, col=6)


    fig.data[2].showscale=False
    fig.data[3].showscale=False



    #fig.add_trace(go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),row=1, col=2)

    f = go.FigureWidget(fig)
    p1 = f.data[1]
    p2 = f.data[2]
    p3 = f.data[3]



    # Hide grey background
    f.update_layout(
        margin=dict(t=0, r=0, b=0, l=0),
        scene=dict(
            xaxis=dict(backgroundcolor="rgba(0,0,0,0)"),
            yaxis=dict(backgroundcolor="rgba(0,0,0,0)"),
            zaxis=dict(backgroundcolor="rgba(0,0,0,0)"),
        ),
        template = "plotly_dark",
        plot_bgcolor = "rgba(0,0,0,0)",
        paper_bgcolor = "rgba(0,0,0,0)",

        xaxis=dict(
                domain=[0.7, 0.75]
            ),
        xaxis2=dict(
                domain=[0.90, 0.95]
            ),
        #xaxis3=dict(
        #        domain=[0.75, 1],
        #),
    )

    f['layout']['yaxis2']['scaleanchor']='x2'
    f['layout']['yaxis']['scaleanchor']='x'



    f.update_layout(
        #width=int(1200*1.2),
        #height=int(700*1.2),
        font_size=7,
        title_font_size=12,
        title={
            'text' : 'Click on a dot to display the corresponding data',
            'y':0.92,
            'x':0.3,
            'xanchor': 'center',
            'yanchor': 'top'}
    )


    #print(f['layout'])
    #f['layout']['xaxis']['domain'] = [0, 0.96]
    #f['layout']['xaxis2']['domain'] = [0.96, 0.98]
    #f['layout']['xaxis3']['domain'] = [0.98, 1.]

    f.layout.hovermode = 'closest'


    # Set background color to zero
    fig.layout.template = "plotly_dark"
    fig.layout.plot_bgcolor = "rgba(0,0,0,0)"
    fig.layout.paper_bgcolor = "rgba(0,0,0,0)"


    # create our callback function
    debug_view = widgets.Output(layout={'border': '1px solid black'})
    @debug_view.capture(clear_output=True)
    def update(trace, points, selector):
        #global p2
        #global p3
        for i in points.point_inds:

            index_init = i
            array_init_genes, l_genes_init = zip(*sorted(zip(array_exp_genes[index_init:index_init+1,:].T, l_genes), reverse=True))
            array_init_genes = np.array(array_init_genes)[:n_to_keep]
            l_genes_init = l_genes_init[:n_to_keep]
            array_init_lipids, l_lipids_init = zip(*sorted(zip(array_exp_lipids[index_init:index_init+1,:].T, l_name_lipids), reverse=True))
            array_init_lipids = np.array(array_init_lipids)[:n_to_keep]
            l_lipids_init = l_lipids_init[:n_to_keep]

            with f.batch_update():

                p2.y = l_genes_init
                p2.z = array_init_genes
                p3.y = l_lipids_init
                p3.z = array_init_lipids

    p1.on_click(update)
f

In [ ]:
if plot_widget:
    debug_view

#### Cluster pixel according to genes and lipids and compare

In [ ]:
print(array_exp_genes.shape, array_exp_lipids.shape)

In [ ]:
# Compute a UMAP projection and do clustering on the result
umap_2d_lipids = UMAP(n_components=2, init='random', random_state=0)
proj_2d_lipids = umap_2d_lipids.fit_transform(array_exp_lipids)
kmeans_lipids = KMeans(n_clusters=100, random_state=0).fit(proj_2d_lipids)

y = pdist(kmeans_lipids.cluster_centers_)
Z = ward(y)
leaves_ordered = list(leaves_list(optimal_leaf_ordering(Z, y)))

l_colors_lipids = []
for label in kmeans_lipids.labels_:
    l_colors_lipids.append(leaves_ordered.index(label))

In [ ]:
fig = go.Figure(data=go.Scattergl(
    x = proj_2d_lipids[:,0],
    y = proj_2d_lipids[:,1],
    mode='markers',
    marker=dict(
        color = l_colors_lipids,
        size=4,
        opacity = 0.4,
        colorscale = px.colors.sequential.Jet,
    )
))


fig.update_traces(textposition='top center', textfont_size=8, showlegend=False)
fig.update_layout(
    width=int(800),
    height=int(800),
    font_size=7,
    title_font_size=12,
    title={
        'text' : 'UMAP and K-Means voxel clustering using lipidomics (MALDI) data',
        'y':0.92,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    template='simple_white'
)
fig.show()
fig.write_image("notebooks/scRNAseq/umap1.pdf")


In [ ]:
# Compute a UMAP projection and do clustering on the result
umap_2d_genes = UMAP(n_components=2, init='random', random_state=0)
proj_2d_genes = umap_2d_genes.fit_transform(array_exp_genes)
kmeans_genes = KMeans(n_clusters=10, random_state=0).fit(proj_2d_genes)

y = pdist(kmeans_genes.cluster_centers_)
Z = ward(y)
leaves_ordered = list(leaves_list(optimal_leaf_ordering(Z, y)))

l_colors_genes = []
for label in kmeans_genes.labels_:
    l_colors_genes.append(leaves_ordered.index(label))
    

In [ ]:
fig = go.Figure(data=go.Scattergl(
    x = proj_2d_genes[:,0],
    y = proj_2d_genes[:,1],
    mode='markers',
    marker=dict(
        color = l_colors_genes,
        size=4,
        opacity = 0.4,
        colorscale = px.colors.sequential.Jet,
    )
))


fig.update_traces(textposition='top center', textfont_size=8, showlegend=False)
fig.update_layout(
    width=int(800),
    height=int(800),
    font_size=7,
    title_font_size=12,
    title={
        'text' : 'UMAP and K-Means voxel clustering using gene (scRNAseq) data',
        'y':0.92,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    template='simple_white'
)
fig.show()
fig.write_image("notebooks/scRNAseq/umap2.pdf")



In [ ]:
fig = go.Figure(data=go.Scattergl(
    x = proj_2d_genes[:,0],
    y = proj_2d_genes[:,1],
    mode='markers',
    marker=dict(
        color = l_colors_lipids,
        size=4,
        opacity = 0.4,
        colorscale = px.colors.sequential.Jet,
    )
))


fig.update_traces(textposition='top center', textfont_size=8, showlegend=False)
fig.update_layout(
    width=int(800),
    height=int(800),
    font_size=7,
    title_font_size=12,
    title={
        'text' : 'UMAP voxel gene data (scRNAseq) projection using Lipid data (MALDI) for clustering',
        'y':0.92,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    template='simple_white'
)

fig.show()
fig.write_image("notebooks/scRNAseq/umap3.pdf")




In [ ]:
fig = go.Figure(data=go.Scattergl(
    x = proj_2d_lipids[:,0],
    y = proj_2d_lipids[:,1],
    mode='markers',
    marker=dict(
        color = l_colors_genes,
        size=4,
        opacity = 0.4,
        colorscale = px.colors.sequential.Jet,
    )
))


fig.update_traces(textposition='top center', textfont_size=8, showlegend=False)
fig.update_layout(
    width=int(800),
    height=int(800),
    font_size=7,
    title_font_size=12,
    title={
        'text' : 'UMAP lipid data (MALDI) projection using gene data (scRNAseq) for clustering',
        'y':0.92,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    template='simple_white'
)

fig.show()
fig.write_image("notebooks/scRNAseq/umap4.pdf")

### Project back cluster in brain space

In [ ]:
if True:
    scatter = go.Scatter3d(
        x=xmol,
        y=zmol,
        z=-ymol,
        mode='markers',
        marker=dict(
            size=1,
            color=l_colors_lipids,
            opacity=1.,
            colorscale = px.colors.sequential.Jet,
        ),
    )
    scatter['hoverinfo']= "all"



    # Get root figure
    root_data = figures._storage.return_shelved_object(
        "figures/3D_page",
        "volume_root",
        force_update=False,
        compute_function=figures.compute_3D_root_volume,
    )

    root_data_y = copy.deepcopy(root_data['y'])
    root_data_z = copy.deepcopy(root_data['z'])
    root_data['y'] = root_data_z 
    root_data['z'] = -root_data_y
    root_data['hoverinfo']= 'skip'



    fig = go.Figure(data=[root_data, scatter])


    # Hide grey background
    fig.update_layout(
        margin=dict(t=0, r=0, b=0, l=0),
            scene=dict(
                xaxis=dict(backgroundcolor="rgba(0,0,0,0)",color="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0)", showticklabels=False, showgrid= False, zeroline = False, showline = False,),
                yaxis=dict(backgroundcolor="rgba(0,0,0,0)",color="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0)", showticklabels=False, showgrid= False, zeroline = False, showline = False,),
                zaxis=dict(backgroundcolor="rgba(0,0,0,0)",color="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0)", showticklabels=False, showgrid= False, zeroline = False, showline = False,),
            ),
            width=1000,
            height=600,
            title={
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'text': 'Lipids clusters (MALDI data)',
                'yanchor': 'top'}
    )

    # Set background color to zero
    fig.layout.template = "plotly_dark"
    fig.layout.plot_bgcolor = "rgba(0,0,0,0)"
    fig.layout.paper_bgcolor = "rgba(0,0,0,0)"

    fig.show()

In [ ]:
if False:
    scatter = go.Scatter3d(
        x=x_mol,
        y=z_mol,
        z=-y_mol,
        mode='markers',
        marker=dict(
            size=1,
            color=kmeans_genes.labels_,    
            opacity=1.,
            colorscale = px.colors.qualitative.T10,
        ),
    )
    scatter['hoverinfo']= "all"

    # Get root figure
    root_data = figures._storage.return_shelved_object(
        "figures/3D_page",
        "volume_root",
        force_update=False,
        compute_function=figures.compute_3D_root_volume,
    )

    root_data_y = copy.deepcopy(root_data['y'])
    root_data_z = copy.deepcopy(root_data['z'])
    root_data['y'] = root_data_z 
    root_data['z'] = -root_data_y


    fig = go.Figure(data=[root_data, scatter])


    # Hide grey background
    fig.update_layout(
        margin=dict(t=0, r=0, b=0, l=0),
            scene=dict(
                xaxis=dict(backgroundcolor="rgba(0,0,0,0)",color="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0)", showticklabels=False, showgrid= False, zeroline = False, showline = False,),
                yaxis=dict(backgroundcolor="rgba(0,0,0,0)",color="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0)", showticklabels=False, showgrid= False, zeroline = False, showline = False,),
                zaxis=dict(backgroundcolor="rgba(0,0,0,0)",color="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0)", showticklabels=False, showgrid= False, zeroline = False, showline = False,),
            ),
            width=1000,
            height=600,
            title={
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'text': 'Genes clusters (scRNAseq data)',
                'yanchor': 'top'},
    )



    # Set background color to zero
    fig.layout.template = "plotly_dark"
    fig.layout.plot_bgcolor = "rgba(0,0,0,0)"
    fig.layout.paper_bgcolor = "rgba(0,0,0,0)"

    fig.show()

In [ ]:
# Function for 3D rotation
def rotate_y(x, y, z, theta):
    w = x+1j*z
    return np.real(np.exp(1j*theta)*w), y, np.imag(np.exp(1j*theta)*w)
    
def animate_figure(fig, output = 'output.gif', multiprocessing = True):
    # Initial coordinates
    x_eye=1.25*0.85
    y_eye=0.1*0.85
    z_eye=1.25*0.85

    # Update with rotation
    time_domain = np.arange(0, 500, 1)
    l_fig = []
    for image, t in enumerate(time_domain):
        fig_rotated = copy.deepcopy(fig)

        # Rotate camera
        xe, ye, ze = rotate_y(x_eye, y_eye, z_eye, -t/40)
        camera = dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=xe, y=ze, z=ye),
        )
        fig_rotated.update_layout(scene_camera=camera)

        # Append final figure to the list of images
        l_fig.append((image,fig_rotated))
    

    def write_figure(t_fig):
        image,fig = t_fig
        fig.write_image("notebooks/scRNAseq/temp/"+ format(image,'04d')+".jpg")
        
    if multiprocessing:
        with Pool(processes=6) as pool:
            [x for x in pool.map(write_figure, l_fig)]
            
    else:
        for image,fig in l_fig:
            print("Plotting figure " + str(image))
            fig.write_image("notebooks/scRNAseq/temp/"+ format(image,'04d')+".jpg")
            
    # Merge images into a gif
    i = "notebooks/scRNAseq/temp/*.jpg"
    o = "notebooks/scRNAseq/temp/" + output
    subprocess.call("convert -delay 2 -loop 0 " + i + " " + o, shell=True)
    
animate_figure(fig, output = 'output_progressive_clustering_lipids.gif', multiprocessing = False)

#### Animation of the most different lipids in 3D

In [ ]:
decrease_resolution_factor = 7
ll_exp_lipids = []
l_name_lipids = []
if True:

    # Simulate a click on all lipid names
    for name in sorted(
        figures._data.get_annotations_MAIA_transformed_lipids(brain_1=True).name.unique()
    ):
        structures = figures._data.get_annotations_MAIA_transformed_lipids(brain_1=True)[
            figures._data.get_annotations_MAIA_transformed_lipids(brain_1=True)["name"] == name
        ].structure.unique()
        for structure in sorted(structures):
            cations = figures._data.get_annotations_MAIA_transformed_lipids(brain_1=True)[
                (
                    figures._data.get_annotations_MAIA_transformed_lipids(brain_1=True)["name"]
                    == name
                )
                & (
                    figures._data.get_annotations_MAIA_transformed_lipids(brain_1=True)[
                        "structure"
                    ]
                    == structure
                )
            ].cation.unique()
            for cation in sorted(cations):
                l_selected_lipids = []
                for slice_index in range(figures._data.get_slice_number()):

                    # Find lipid location
                    l_lipid_loc = (
                        figures._data.get_annotations()
                        .index[
                            (figures._data.get_annotations()["name"] == name)
                            & (figures._data.get_annotations()["structure"] == structure)
                            & (figures._data.get_annotations()["slice"] == slice_index)
                            & (figures._data.get_annotations()["cation"] == cation)
                        ]
                        .tolist()
                    )

                    # If several lipids correspond to the selection, we have a problem...
                    if len(l_lipid_loc) > 1:
                        logging.warning("More than one lipid corresponds to the selection")
                        l_lipid_loc = [l_lipid_loc[-1]]
                    # If no lipid correspond to the selection, set to -1
                    if len(l_lipid_loc) == 0:
                        l_lipid_loc = [-1]

                    # add lipid index for each slice
                    l_selected_lipids.append(l_lipid_loc[0])

                # Get final lipid name
                lipid_string = name + " " + structure + " " + cation

                # If lipid is present in at least one slice
                if np.sum(l_selected_lipids) > -figures._data.get_slice_number():

                    # Build the list of mz boundaries for each peak and each index
                    lll_lipid_bounds = [
                        [
                            [
                                (
                                    float(figures._data.get_annotations().iloc[index]["min"]),
                                    float(figures._data.get_annotations().iloc[index]["max"]),
                                )
                            ]
                            if index != -1
                            else None
                            for index in [lipid_1_index, -1, -1]
                        ]
                        for lipid_1_index in l_selected_lipids
                    ]
                    print("getting data for lipid ", lipid_string)
                    
                    l_expr = []

                    try:
                        interpolated_array = figures.compute_3D_volume_figure(ll_t_bounds = lll_lipid_bounds, name_lipid_1=lipid_string,decrease_dimensionality_factor=decrease_resolution_factor,return_interpolated_array=True, structure_guided_interpolation=True, divider_radius=12)
                    except Exception as e:
                        print(e)
                    
                    l_name_lipids.append(lipid_string)
                    ll_exp_lipids.append(interpolated_array)



    array_exp_lipids_for_display = np.array(ll_exp_lipids)
    
    with open('notebooks/scRNAseq/data/array_exp_lipids_for_display.npy', 'wb') as f:
        np.save(f, array_exp_lipids_for_display)

In [ ]:
print(array_exp_lipids_for_display.shape)

In [ ]:
l_array_exp_lipids_for_display_flat = []
for arr in array_exp_lipids_for_display:
    l_array_exp_lipids_for_display_flat.append(arr.flatten())
array_exp_lipids_for_display_flat = np.array(l_array_exp_lipids_for_display_flat)
l_array_exp_lipids_for_display_flat = None

In [ ]:
# Compute a UMAP projection and do clustering on the result
umap_2d_all_lipids = UMAP(n_components=2, init='random', random_state=0)
proj_2d_all_lipids = umap_2d_all_lipids.fit_transform(array_exp_lipids_for_display_flat)
kmeans_all_lipids = KMeans(n_clusters=len(proj_2d_all_lipids), random_state=0).fit(proj_2d_all_lipids)

y = pdist(kmeans_all_lipids.cluster_centers_)
Z = ward(y)
leaves_ordered = list(leaves_list(optimal_leaf_ordering(Z, y)))

l_colors_all_lipids = []
for label in kmeans_all_lipids.labels_:
    l_colors_all_lipids.append(leaves_ordered.index(label))
    
    
fig = go.Figure(data=go.Scattergl(
    x = proj_2d_all_lipids[:,0],
    y = proj_2d_all_lipids[:,1],
    mode='markers',
    text = [str(i) + '\n\n' + x for i, x in enumerate(l_name_lipids)],
    marker=dict(
        color = l_colors_all_lipids,
        size=4,
        opacity = 0.4,
        colorscale = px.colors.sequential.Jet,
    )
))


fig.update_traces(textposition='top center', textfont_size=8, showlegend=False)
fig.update_layout(
    width=int(800),
    height=int(800),
    font_size=7,
    title_font_size=12,
    title={
        'text' : 'UMAP and K-Means voxel clustering using lipidomics (MALDI) data',
        'y':0.92,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    template='simple_white'
)
fig.show()
fig.write_image("notebooks/scRNAseq/umap_all_lipids.pdf")

In [ ]:
@njit
def median_symmetric_filter(array_lipid):
    size_filter = 1
    array_lipid_filtered = np.zeros_like(array_lipid)
    for slice_index in range(int(array_lipid.shape[2]/2)+1):
        for x in range(size_filter,array_lipid.shape[0]-size_filter):
            for y in range(size_filter,array_lipid.shape[1]-size_filter):
                # Compute median in a square of size 2*size_filter+1
                median = np.median(np.vstack((array_lipid[x-size_filter:x+size_filter+1, y-size_filter:y+size_filter+1, slice_index], array_lipid[x-size_filter:x+size_filter+1, y-size_filter:y+size_filter+1, -slice_index])))
                array_lipid_filtered[x,y,slice_index] = median
                array_lipid_filtered[x,y,-slice_index] = median
                
    if array_lipid.shape[2]%2!=0:
        array_lipid_filtered[:,:,int(array_lipid.shape[2]/2)+1] = array_lipid[:,:,int(array_lipid.shape[2]/2)+1]
    return array_lipid_filtered
                

In [ ]:
# Get lipid indexes to plot
lipid_1 = 36
lipid_2 = 25
lipid_3 = 50

# Compute a median symmetric filter
array_lipid_1 = median_symmetric_filter(array_exp_lipids_for_display[lipid_1])
array_lipid_2 = median_symmetric_filter(array_exp_lipids_for_display[lipid_2])
array_lipid_3 = median_symmetric_filter(array_exp_lipids_for_display[lipid_3])

# Initial coordinates
x_eye=1.25*0.85
y_eye=0.1*0.85
z_eye=1.25*0.85
    

# Get the corresponding coordinates
shape_x, shape_y, shape_z = array_exp_lipids_for_display.shape[1:]
X, Y, Z = np.mgrid[
    0 : shape_x
    / 1000
    * 25
    * decrease_resolution_factor : shape_x
    * 1j,
    0 : shape_y
    / 1000
    * 25
    * decrease_resolution_factor : shape_y
    * 1j,
    0 : shape_z
    / 1000
    * 25
    * decrease_resolution_factor : shape_z
    * 1j,
]

if True:
    
    # Compute first maldi lipid in 3D
    volume_maldi = go.Volume(
        x=X.flatten(),
        y=Z.flatten(),
        z=-Y.flatten(),
        value=array_lipid_1.flatten(),
        isomin=0.01,
        isomax=1.5,
        opacity = 0,
        opacityscale='uniform',
        surface_count=10,  # needs to be a large number for good volume rendering
        colorscale="Reds",
    )

    # Compute second maldi lipid in 3D
    volume_maldi_2 = go.Volume(
        x=X.flatten(),
        y=Z.flatten(),
        z=-Y.flatten(),
        value=array_lipid_2.flatten(),
        isomin=0.01,
        isomax=1.5,
        opacity = 0,
        opacityscale='uniform',
        surface_count=10,  # needs to be a large number for good volume rendering
        colorscale="Blues",
    )
    
    # Compute thid maldi lipid in 3D
    volume_maldi_3 = go.Volume(
        x=X.flatten(),
        y=Z.flatten(),
        z=-Y.flatten(),
        value=array_lipid_3.flatten(),
        isomin=0.01,
        isomax=1.5,
        opacity = 0,
        opacityscale='uniform',
        surface_count=10,  # needs to be a large number for good volume rendering
        colorscale="Greens",
    )



    # Get root figure
    root_data = figures._storage.return_shelved_object(
        "figures/3D_page",
        "volume_root",
        force_update=False,
        compute_function=figures.compute_3D_root_volume,
    )

    root_data_y = copy.deepcopy(root_data['y'])
    root_data_z = copy.deepcopy(root_data['z'])
    root_data['y'] = root_data_z 
    root_data['z'] = -root_data_y
    root_data['hoverinfo']= 'skip'
    
    

    # Define figure and layout
    fig = go.Figure(data=[root_data, volume_maldi, volume_maldi_2, volume_maldi_3])

    # Transparent background
    fig.update_layout(
        margin=dict(t=0, r=0, b=0, l=0),
        scene=dict(
            xaxis=dict(backgroundcolor="rgba(0,0,0,0)",color="white", gridcolor="white", showticklabels=False),
            yaxis=dict(backgroundcolor="rgba(0,0,0,0)",color="white", gridcolor="white", showticklabels=False),
            zaxis=dict(backgroundcolor="rgba(0,0,0,0)",color="white", gridcolor="white", showticklabels=False),
        ),
        # Bigger size for nicer display
        width=1000,
        height=600,
    )
    fig.update_layout(
        title={
            'y':0.94,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'}
    )

    # No colorbar
    fig.update_traces(showscale=False)

    #fig.show()
    #fig.write_image("notebooks/scRNAseq/temp/test.jpg")
    
    if True:
        # Update with rotation
        t1 = 0
        t2 = 200
        t3 = 250
        t4 = 450
        t5 = 500
        t6 = 700
        time_domain = np.arange(t1, t4, 1)


        l_fig = []
        for image, t in enumerate(time_domain):
            fig_rotated = copy.deepcopy(fig)

            # Remove reference always after t=150, and display data always
            if t>=t1 and t<t2:
                # Title
                fig_rotated.update_layout(title_text=l_name_lipids[lipid_1])
                # First lipid is displayed fully
                fig_rotated.data[1].opacity = 1

            # Switch between two lipids
            elif t>=t2 and t<t3:
                # No more title as switch taking place
                fig_rotated.update_layout(title_text='')

                # Fade away lipid 1
                fig_rotated.data[1].opacity = (t3-t)/(t3-t2)
                # Progressivelu show lipid 2
                fig_rotated.data[2].opacity = (t-t2)/(t3-t2)

            # Just leave lipid 2
            elif t>=t3:# and t<t4:
                # Title
                fig_rotated.update_layout(title_text=l_name_lipids[lipid_2])
                # Lipid 2 is displayed fully
                fig_rotated.data[2].opacity = 1

#            # Progressively display lipid 3
#            elif t>=t4 and t<t5:
#                # No more title as switch taking place
#                fig_rotated.update_layout(title_text='')
#                # Lipid 2 fades away
#                fig_rotated.data[2].opacity = (t5-t)/(t5-t4)
#                # Lipid 3 appears
#                fig_rotated.data[3].opacity = (t-t4)/(t5-t4)
#
#            # Just leave new lipid
#            elif t>=t5:
#                # Title
#                fig_rotated.update_layout(title_text=l_name_lipids[lipid_3])
#
#                # Lipid 3 is displayed fully
#                fig_rotated.data[3].opacity = 1

            # Rotate camera
            xe, ye, ze = rotate_y(x_eye, y_eye, z_eye, -t/40)
            camera = dict(
                up=dict(x=0, y=0, z=1),
                center=dict(x=0, y=0, z=0),
                eye=dict(x=xe, y=ze, z=ye),
            )
            fig_rotated.update_layout(scene_camera=camera)

            # Append final figure to the list of images
            l_fig.append((image,fig_rotated))

In [ ]:
output = 'two_lipids.gif'

def write_figure(t_fig):
    image,fig = t_fig
    fig.write_image("notebooks/scRNAseq/temp/"+ format(image,'04d')+".jpg")

multiprocessing = True
if multiprocessing:
    with Pool(processes=10) as pool:
        print("Wallah")
        [x for x in pool.map(write_figure, l_fig)]

else:
    for image,fig in l_fig:
        print("Plotting figure " + str(image))
        fig.write_image("notebooks/scRNAseq/temp/"+ + format(image,'04d')+".jpg")



In [ ]:
# Merge images into a gif
i = "notebooks/scRNAseq/temp/*.jpg"
o = "notebooks/scRNAseq/temp/" + output
subprocess.call("convert -delay 3 -loop 0 " + i + " " + o, shell=True)